In [1]:
!pip install pandas
!pip install matplotlib
!pip install plotly

In [14]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam

import tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers import dp_optimizer_keras

from synthesizers.cgan.model import ConditionalGAN, GANMonitor
from synthesizers.preprocessing.wesad import Subject, WESADDataset
from synthesizers.utils.training import data_split, buildDataCTST, generate_and_plot_data, synthetic_dataset


In [3]:
DATA_PATH = 'data/wesad/wesad_preprocessed_1hz.csv'
SAMPLING_RATE = 1

In [4]:
df = pd.read_csv(DATA_PATH, index_col=0)
df_label = df['label']
df_stress = df[df['label']==1]
df_no_stress = df[df['label']==0]

In [5]:
trainX, label_trainX = WESADDataset.create_windows(df, 1)
mos, _ = WESADDataset.create_windows(df_stress, SAMPLING_RATE)
non_mos, _ = WESADDataset.create_windows(df_no_stress,SAMPLING_RATE)

In [6]:
num_split = 0.8
trainmos, testmos = data_split(mos, num_split)
trainnomos, testnomos = data_split(non_mos, num_split)

print(trainmos.shape)
print(testmos.shape)
print(trainnomos.shape)
print(testnomos.shape)

(132, 60, 7)
(34, 60, 7)
(308, 60, 7)
(78, 60, 7)


In [7]:
# Training Hyperparameters
NUM_FEATURES = trainX.shape[2]
SEQ_LENGTH = 60
LATENT_DIM = SEQ_LENGTH
BATCH_SIZE = 32
HIDDEN_UNITS = 16
EPOCHS = 10

# load dataset into tf dataset
dataset = tf.data.Dataset.from_tensor_slices((trainX, label_trainX))
dataset = dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 60, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [8]:
randomTrainMos = tf.random.normal(shape=(trainmos.shape[0], LATENT_DIM))
randomTrainNoMos = tf.random.normal(shape=(trainnomos.shape[0], LATENT_DIM))
randomTestMos = tf.random.normal(shape=(testmos.shape[0], LATENT_DIM))
randomTestNoMos = tf.random.normal(shape=(testnomos.shape[0], LATENT_DIM))

In [17]:
l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
    )


In [20]:
cond_gan = ConditionalGAN(
    num_features=NUM_FEATURES,
    seq_length=SEQ_LENGTH,
    latent_dim=LATENT_DIM,
    discriminator=ConditionalGAN.conditional_discriminator(
        seq_length=SEQ_LENGTH, 
        num_features=NUM_FEATURES), 
    generator=ConditionalGAN.conditional_generator(
        hidden_units=HIDDEN_UNITS, 
        seq_length=SEQ_LENGTH, 
        latent_dim=LATENT_DIM,
        num_features=NUM_FEATURES
    )
)

cond_gan.compile(
    d_optimizer= optimizer, #Adam(learning_rate=0.0002, beta_1=0.5), #optimizer
    g_optimizer=optimizer, #Adam(learning_rate=0.0002, beta_1=0.5), #optimizer
    loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
)

history = cond_gan.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[
        GANMonitor(
            trainmos,
            trainnomos,
            testmos,
            testnomos,
            randomTrainMos,
            randomTrainNoMos,
            randomTestMos,
            randomTestNoMos,
            num_seq=50,
            seq_length=SEQ_LENGTH,
            num_features=NUM_FEATURES,
        )
    ],
)

cond_gan.generator.save(f"models/cond_generator_{EPOCHS}")
cond_gan.discriminator.save(f"models/cond_discriminator_{EPOCHS}")


Epoch 1/10


AssertionError: in user code:

    File "/Users/nils/miniconda3/envs/cgan_clean/lib/python3.8/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/nils/miniconda3/envs/cgan_clean/lib/python3.8/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/nils/miniconda3/envs/cgan_clean/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/nils/thesis/Data_Generation/synthesizers/cgan/model.py", line 216, in train_step
        self.d_optimizer.apply_gradients(
    File "/Users/nils/miniconda3/envs/cgan_clean/lib/python3.8/site-packages/tensorflow_privacy/privacy/optimizers/dp_optimizer_keras.py", line 230, in apply_gradients
        assert self._was_dp_gradients_called, (

    AssertionError: Neither _compute_gradients() or